Importação bibliotecas

In [1]:
import pandas as pd # Fundamental para análise e manipulação de dados
import requests # Para fazer requisições HTTP (acessar APIs, baixar dados da web)
import os # Para interagir com o sistema operacional (manipular arquivos, pastas)
from datetime import datetime # Módulo para trabalhar com datas e horas
import yfinance as yf # Baixar dados do mercado financeiro (Yahoo Finance)
from pytrends.request import TrendReq # Extrair dados de tendências de busca (Google Trends)
from sidrapy import get_table # Função específica para buscar dados de tabelas do SIDRA (IBGE)

Função de Coleta do Banco Central (SGS)

- Busca os indicadores e salva em um arquivo CSV na pasta data_raw

In [2]:
# Variáveis centrais
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y/%m/%d')
data_raw_path = '../data_raw/'
# Se existir a pasta ok, se não é criada
os.makedirs(data_raw_path, exist_ok=True)

# Definição {dicionário} de indicadores BCB
indicadores_bcb = {
    '432': 'selic_meta',             # SELIC (Meta) - Mensal
    '433': 'ipca_mensal',            # IPCA (Var. % mensal) - Mensal
    '4393': 'icc_consumidor_mensal', # Índice de Confiança do Consumidor - Mensal
    '24364': 'pib_mensal'            # Produto interno bruto - Mensal
}

def dados_bc (codigo_indicador, nome_indicador):

    start_date_fmt = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d/%m/%Y')
    end_date_fmt = datetime.now().strftime('%d/%m/%Y')

    url_api = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_indicador}/dados?formato=json&dataInicial={start_date_fmt}&dataFinal={end_date_fmt}"
    
    #Chamar a API
    response = requests.get(url_api)
    
    if response.status_code == 200: #Código 200 significa êxito
        dados_json = response.json()
        
        if not dados_json: # Check se a API retornar sem dados
            print(f"   > AVISO: Série {nome_indicador} não retornou dados.")
            return

        # Converte para DataFrame
        df = pd.DataFrame(dados_json)
        
        # Renomeia a coluna 'valor' para o nome do indicador
        df = df.rename(columns={'valor': nome_indicador})
        
        # Salvar o arquivo CSV individual (Criando novas pastas)
        file_path = os.path.join(data_raw_path, f'{nome_indicador}.csv')
        df.to_csv(file_path, index=False)
        
        print(f"   > Sucesso. Salvo em: {file_path}")
        
    else:
        print(f"   > ERRO! Falha ao buscar {nome_indicador}. Status: {response.status_code}")

# Loop para a função de coleta BCB
for codigo, nome in indicadores_bcb.items():
    dados_bc(codigo, nome)

   > Sucesso. Salvo em: ../data_raw/selic_meta.csv
   > Sucesso. Salvo em: ../data_raw/ipca_mensal.csv
   > Sucesso. Salvo em: ../data_raw/icc_consumidor_mensal.csv
   > Sucesso. Salvo em: ../data_raw/pib_mensal.csv


Coleta Taxa de Desocupação (IBGE/SIDRA)

In [3]:
try:
    desemprego_df = get_table(
        table_code='4095',
        territorial_level='1',
        ibge_territorial_code='all',
        variable='4099',
        period='all' 
    )
    
    if not desemprego_df.empty:
        desemprego_df = desemprego_df.loc[1:, ['V', 'D2C']]
        desemprego_df.columns = ['taxa_desocupacao', 'periodo_codigo_trimestre']
        desemprego_df['taxa_desocupacao'] = pd.to_numeric(desemprego_df['taxa_desocupacao'], errors='coerce')
        # Filtro de data
        desemprego_df = desemprego_df[desemprego_df['periodo_codigo_trimestre'] >= '202001']
        
        file_path = os.path.join(data_raw_path, 'desemprego_trimestral.csv')
        desemprego_df.to_csv(file_path, index=False)
        print(f"   > Sucesso. Salvo em: {file_path}")
    else:
        print("   > AVISO: Dados de desemprego não retornaram.")
        
except Exception as e:
    print(f"   > ERRO GERAL! Falha ao buscar Taxa de Desocupação : {e}")

   > Sucesso. Salvo em: ../data_raw/desemprego_trimestral.csv


Coleta de Dados das Empresas (Yahoo Finance)

In [ ]:
# Dicionário com os Tickers 
tickers = {
    'MGLU3': 'magalu',
    'MELI': 'mercado_livre', # Ticker NASDAQ
    'BHIA3': 'casas_bahia'
}

# Lista VAZIA para guardar os DataFrames de cada empresa
lista_dfs_empresas = []

# 3. Loop para buscar os dados de cada empresa
for ticker, nome in tickers.items():

    # --- Monta o ticker completo ---
    if ticker != 'MELI':
        ticker_completo = ticker + '.SA'
    else:
        ticker_completo = ticker

    try:
        empresa = yf.Ticker(ticker_completo)
        dre_trimestral = empresa.quarterly_financials

        if not dre_trimestral.empty:
            # Transpõe a tabela (T) - Datas viram linhas
            dre_trimestral_t = dre_trimestral.T
            # reset_index() transforma o índice (data) em uma coluna 'data'
            dre_com_data_coluna = dre_trimestral_t.reset_index()
            # Adiciona a coluna 'empresa' com o nome atual do loop
            dre_com_data_coluna['empresa'] = nome
            # Adiciona o DataFrame à lista vazia
            lista_dfs_empresas.append(dre_com_data_coluna)

            print(f"   > Dados de {nome} coletados e adicionados à lista.")
        else:
             print(f"   > AVISO: Não foram encontrados dados financeiros para {nome} ({ticker_completo}).")

    except Exception as e:
        print(f"   > ERRO GERAL! Falha ao buscar dados de {nome}: {e}")

# Verifica se a lista não está vazia
if lista_dfs_empresas:
    # Concatena todos os DataFrames da lista
    df_empresas_consolidado = pd.concat(lista_dfs_empresas, ignore_index=True) # ignore_index=True cria um novo índice numérico para o DataFrame final
    cols = df_empresas_consolidado.columns.to_list()
    cols.remove('empresa')
    new_order = ['empresa'] + cols
    df_empresas_consolidado = df_empresas_consolidado[new_order]

    # Salvar o DataFrame único
    file_path_consolidado = os.path.join(data_raw_path, 'empresas_dre_trimestral.csv')
    df_empresas_consolidado.to_csv(file_path_consolidado, index=False)

else:
    print("\n   > AVISO: Nenhum dado de empresa foi coletado. Arquivo consolidado não gerado.")

   > Dados de magalu coletados e adicionados à lista.
   > Dados de mercado_livre coletados e adicionados à lista.
   > Dados de casas_bahia coletados e adicionados à lista.


Coleta Google Trends para as empresas

In [5]:
pytrends = TrendReq(hl='pt-BR', tz=360)

keywords = ['Magazine Luiza', 'Casas Bahia', 'Mercado Livre']

pytrends.build_payload(keywords, timeframe='today 5-y',geo='BR')

df = pytrends.interest_over_time()

if not df.empty:
    if 'isPartial' in df.columns:
        df = df.drop(columns=['isPartial'])

    file_path = os.path.join(data_raw_path, 'gtrends.csv')
    df.to_csv(file_path, index_label='data')
    print(f"   > Sucesso. Salvo Google Trends em: {file_path}")
else:
    print("   > AVISO: Dados do Google Trends não retornaram.")

   > Sucesso. Salvo Google Trends em: ../data_raw/gtrends.csv


c:\Users\Thiag\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
